In [144]:
# 파라미터 설정
dsm_file_path = r"C:\Users\kdw\Desktop\python_work_space\EAP\12\dsm.jpg"  # dsm 이미지 경로
image_path = r"C:\Users\kdw\Desktop\python_work_space\EAP\12\mask.png" # mask 이미지 경로
org_file_path=r"C:\Users\kdw\Desktop\python_work_space\EAP\12\org.jpg" # 원본 이미지 경로
min_altitude = 125.0
max_altitude = 165.0
# 시작점
start = (428, 66)

In [146]:
import os
import numpy as np
import networkx as nx
from PIL import Image, ImageDraw
import math
import queue

# 이미지에서 길을 찾는 함수
def find_roads(image_path, road_colors):
    image = Image.open(image_path)
    image_array = np.array(image)

    road_pixels = np.zeros(image_array.shape[:2], dtype=bool)
    for color in road_colors:
        road_pixels |= np.all(image_array[:, :, :3] == color, axis=-1)

    return np.argwhere(road_pixels)

# 주어진 모든 도로 픽셀에 대해 그래프를 생성합니다.
def create_graph(road_pixels):
    G = nx.Graph()
    road_pixel_set = set(map(tuple, road_pixels))  # 도로 픽셀을 셋으로 변환

    for road_pixel in road_pixel_set:  # road_pixels 내의 각 픽셀에 대해
        G.add_node(road_pixel)
        for neighbor in [(0, 1), (1, 0), (0, -1), (-1, 0)]:  # 상하좌우 이웃
            neighbor_pixel = tuple(np.array(road_pixel) + np.array(neighbor))
            if neighbor_pixel in road_pixel_set:  # 이웃이 도로 픽셀인지 확인
                G.add_edge(road_pixel, neighbor_pixel, weight=1)
    return G

def find_nearest_road(point, road_pixels, image_size):
    road_pixel_set = set(map(tuple, road_pixels))
    visited = set()
    q = queue.Queue()

    # 시작점을 큐에 추가
    q.put(point)
    visited.add(point)

    while not q.empty():
        current_pixel = q.get()

        # 현재 픽셀이 도로 픽셀이라면 반환
        if current_pixel in road_pixel_set:
            return current_pixel

        # 상하좌우 이웃 픽셀을 큐에 추가
        for dy, dx in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            neighbor_pixel = (current_pixel[0] + dy, current_pixel[1] + dx)

            # 이미지 범위를 벗어나지 않고, 방문하지 않은 픽셀인 경우
            if 0 <= neighbor_pixel[0] < image_size[0] and 0 <= neighbor_pixel[1] < image_size[1]:
                if neighbor_pixel not in visited:
                    q.put(neighbor_pixel)
                    visited.add(neighbor_pixel)

    return None  # 도로 픽셀을 찾지 못한 경우

# 고도 데이터를 불러오는 함수
def load_altitude_data(file_path):
    dsm_image = Image.open(file_path)
    dsm_array = np.array(dsm_image.convert('L'))
    normalized_dsm_array = dsm_array / 255.0
    altitude_array = (normalized_dsm_array * (max_altitude - min_altitude)) + min_altitude
    return altitude_array

def connect_point_to_road(G, point, road_pixels):
    nearest_road_pixel = find_nearest_road(point, road_pixels, image_size)
    if nearest_road_pixel is not None:
        G.add_node(point)
        G.add_edge(point, tuple(nearest_road_pixel), weight=1)


# 도로로 간주할 여러 색상을 리스트에 추가
road_colors = [
    [255, 255, 0],  # 노란색
    [192, 192, 0],  # 올리브색
    [0, 255, 0],    # 녹색
    [128, 128, 128] # 회색
]

image = Image.open(image_path)
image_array = np.array(image)
image_size = image_array.shape[:2]
# 길 찾기
road_pixels = find_roads(image_path, road_colors)
G = create_graph(road_pixels)
# 고도 데이터 불러오기
altitude_data = load_altitude_data(dsm_file_path)

if start not in G:
    connect_point_to_road(G, start, road_pixels)



In [147]:
import networkx as nx

# 이미지와 고도 데이터 불러오기
image = Image.open(image_path)  # image_path를 이미지 파일 경로로 설정해야 합니다.
image_array = np.array(image)
altitude_data = load_altitude_data(dsm_file_path)  # 고도 데이터 파일 경로 설정 필요

# start 위치에서 도달할 수 있는 모든 노드의 경로 길이를 찾습니다.
reachable_nodes = nx.single_source_shortest_path_length(G, start)

def find_highest_altitude_among_reachable(image_array, altitude_data, reachable_nodes):
    num_rows, num_cols = image_array.shape[:2]
    highest_altitude = -np.inf  # 초기 가장 높은 고도를 매우 낮은 값으로 설정
    highest_point = None  # 가장 높은 고도의 픽셀 위치

    for y in range(num_rows):
        for x in range(num_cols):
            if (y, x) in reachable_nodes and not np.array_equal(image_array[y, x, :3], [255, 0, 0]):
                altitude = altitude_data[y, x]
                if altitude > highest_altitude:
                    highest_altitude = altitude
                    highest_point = (y, x)

    return highest_altitude, highest_point

# start와 연결된 점 중에서 가장 높은 고도 찾기
highest_altitude, highest_point = find_highest_altitude_among_reachable(image_array, altitude_data, reachable_nodes)
print(f"start와 연결된 점 중 빨간색이 아닌 픽셀 중 가장 높은 고도: {highest_altitude} at {highest_point}")


start와 연결된 점 중 빨간색이 아닌 픽셀 중 가장 높은 고도: 146.1764705882353 at (1394, 2270)


In [178]:
#시작점의 고도 찾기

def get_altitude_at_pixel(altitude_data, pixel):
    y, x = pixel
    if 0 <= y < altitude_data.shape[0] and 0 <= x < altitude_data.shape[1]:
        return altitude_data[y, x]
    else:
        return None  # 픽셀이 데이터 범위를 벗어나는 경우

# 고도 데이터 불러오기
altitude_data = load_altitude_data(dsm_file_path)  # 고도 데이터 파일 경로 설정 필요

# 특정 픽셀의 고도 확인
pixel = (1300, 1800) # 픽셀 위치
altitude = get_altitude_at_pixel(altitude_data, pixel)

if altitude is not None:
    print(f"픽셀 {pixel}의 고도: {altitude}")
else:
    print(f"픽셀 {pixel}은 데이터 범위 밖에 있습니다.")


픽셀 (1300, 1800)의 고도: 135.66666666666666


In [18]:
import os
import numpy as np
import networkx as nx
from PIL import Image, ImageDraw
import math
import queue
from skimage.measure import label, regionprops
from skimage.morphology import dilation, square

def find_nearest_road(point, road_pixels, image_size):
    road_pixel_set = set(map(tuple, road_pixels))
    visited = set()
    q = queue.Queue()

    # 시작점을 큐에 추가
    q.put(point)
    visited.add(point)

    while not q.empty():
        current_pixel = q.get()

        # 현재 픽셀이 도로 픽셀이라면 반환
        if current_pixel in road_pixel_set:
            return current_pixel

        # 상하좌우 이웃 픽셀을 큐에 추가
        for dy, dx in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
            neighbor_pixel = (current_pixel[0] + dy, current_pixel[1] + dx)

            # 이미지 범위를 벗어나지 않고, 방문하지 않은 픽셀인 경우
            if 0 <= neighbor_pixel[0] < image_size[0] and 0 <= neighbor_pixel[1] < image_size[1]:
                if neighbor_pixel not in visited:
                    q.put(neighbor_pixel)
                    visited.add(neighbor_pixel)

    return None  # 도로 픽셀을 찾지 못한 경우

def is_not_red(pixel, image_array):
    # 이미지 배열의 크기 확인
    num_rows, num_cols = image_array.shape[:2]

    # 픽셀 좌표가 이미지 배열 범위 내에 있는지 확인
    if 0 <= pixel[0] < num_rows and 0 <= pixel[1] < num_cols:
        return not np.array_equal(image_array[pixel[0], pixel[1], :3], [255, 0, 0])
    else:
        return False  # 픽셀 좌표가 범위를 벗어나면 False 반환

def find_target(altitude_data, goal_height, G, start, image_path):

    if altitude_data[start] >= goal_height:
        print("고도가 높아 경로 탐색이 필요 없습니다.")
        return None
    
    image = Image.open(image_path)
    image_array = np.array(image)

    # 고도가 goal_height 이상인 픽셀들 중 최소 고도를 찾는다
    mask = altitude_data >= goal_height
    min_altitude = np.min(altitude_data[mask])

    # 최소 고도를 가진 픽셀의 좌표를 찾는다
    potential_targets = np.argwhere((altitude_data == min_altitude) & mask)

    # 출발점에서 가장 가까운 픽셀을 찾는다
    min_distance = np.inf
    target = None
    for y, x in potential_targets:
        if is_not_red((y, x), image_array) and (y, x) in G:
            try:
                path = nx.shortest_path(G, source=start, target=(y, x), weight='weight')
                distance = len(path)
                if distance < min_distance:
                    min_distance = distance
                    target = (y, x)
            except nx.NetworkXNoPath:
                continue

    return target


image = Image.open(image_path)
image_array = np.array(image)
image_size = image_array.shape[:2]

goal_height = 143

target = find_target(altitude_data, goal_height, G, start, image_path)
print(target)


(2204, 2037)


In [177]:
import os
import numpy as np
import networkx as nx
from PIL import Image, ImageDraw
from heapq import heappush, heappop
import math

def heuristic(a, b):
    # 유클리드 거리를 휴리스틱으로 사용합니다.
    return np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def find_path(img, start, goal, road_colors):

    # 이미지를 로드하고 numpy 배열로 변환합니다.
    data = np.array(img)

    # 색상을 유지할 마스크를 생성합니다.
    road_mask = np.zeros(data.shape[:2], dtype=bool)
    for color in road_colors:
        road_mask |= np.all(data[:, :, :3] == color, axis=-1)

    # A* 알고리즘을 사용하여 최단 경로를 찾습니다.
    path = nx.astar_path(G, start, goal, heuristic)

    return path

# 경로를 이미지에 그리는 함수
def draw_path_on_image(org_path, paths, line_colors, line_width):
    original_image = Image.open(org_path)
    draw = ImageDraw.Draw(original_image)

    for path, line_color in zip(paths, line_colors):
        for i in range(len(path)-1):
            draw.line((path[i][1], path[i][0], path[i+1][1], path[i+1][0]), fill=line_color, width=line_width)

    output_path = org_path.replace('.jpg', '_path.jpg')
    original_image.save(output_path)
    original_image.show()
    return output_path


img = Image.open(image_path)
# 'start'에서 'target'까지의 경로를 찾는다
shortest_path_to_target = find_path(img, start, target, road_colors)
print(len(shortest_path_to_target))
# 'target'에서 대피소까지의 경로를 찾는다
shortest_path_from_target = find_path(img, target, (1520,2100), road_colors)

# 두 경로를 이미지에 그린다
paths = [shortest_path_to_target, shortest_path_from_target]
colors = ['red', 'orange']  # 빨간색과 주황색으로 설정
image_with_path = draw_path_on_image(org_file_path, paths, colors, 10)



2289
